In [34]:
# to do


# capture frame
# in a thread -> detect frame 
# in a thread -> track body
# in a function (at the end)-> final output





In [1]:
import numpy as np
import cv2 as cv
import torch
from facenet_pytorch import MTCNN
import pickle
from keras_facenet import FaceNet
from threading import Thread
from datetime import timedelta
import datetime
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
import torch
print(torch.cuda.is_available())
import os
import sys
import queue
print("OpenCL Available:", cv.ocl.haveOpenCL())
print("OpenCL in Use:", cv.ocl.useOpenCL())
sys.path.append(r"D:\Nexus AI\Nexus Face recognition\sort")
sort_path = os.path.join(os.getcwd(), 'sort')
sys.path.insert(0, sort_path)
from sort import Sort
import datetime
from threading import Event
import time
from config import RTSP_URL
from ultralytics import YOLO
from logging.handlers import RotatingFileHandler
import logging
from logging.handlers import RotatingFileHandler
import os
stop_event = Event()
from queue import Queue

[]
False
OpenCL Available: True
OpenCL in Use: True


In [2]:
# Queus for threading
frame_queue = Queue(maxsize = 30) 
face_crop_queue = Queue(maxsize = 20)
may_be_queue = Queue(maxsize = 20)

In [3]:
# models
ymodel = YOLO("yolov8n.pt")

with open('model/svm_model_160x160.pkl','rb') as f:
    model = pickle.load(f)

with open('model/label_encoder.pkl','rb') as g:
    encoder = pickle.load(g)

    
tracker = Sort(max_age=5, min_hits=3, iou_threshold=0.2)

embedder = FaceNet()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(keep_all=True, device=device)

C:\Python compiler\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator SVC from version 1.6.1 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Python compiler\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
# logs
log_formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
log_file = 'logs/attendance_logs.log'
file_handler = RotatingFileHandler(log_file, maxBytes=5*1024*1024, backupCount=3)  # 5MB per file, keep 3 backups
file_handler.setFormatter(log_formatter)
console_handler = logging.StreamHandler()
console_handler.setFormatter(log_formatter)
logging.basicConfig(level=logging.INFO, handlers=[file_handler, console_handler])
logger = logging.getLogger()

In [5]:
# global varaibels

track_id_to_name = {}
in_time_log = {}
out_time_log = {}
face_recognition_attempts = {}  # track_id -> count # track_id -> name
batch_id = []
already_recognized =[]
employees = {'parn':False, 'Adivya':False}
timeduration = {'parn':[], 'Adivya':[]}
newemployee = {'parn':[],'Adivya':[]}
BATCH_SIZE = 5
last_seen_frame = {}
frame_count = 0
inactive_threshold = 5
new_width = 1280
new_height = 720

In [ ]:
# functions

def get_batch_embeddings(face_batch):
    # Convert the list of face images to a NumPy array and ensure they are float32
    face_batch = np.array([face.astype("float32") for face in face_batch])
    embeddings = embedder.embeddings(face_batch)  # Predict embeddings for the batch
    return embeddings  # Return the embeddings for all faces in the batch



class CameraStream:
    def __init__(self, rtsp_url):
        self.rtsp_url = rtsp_url
        self.cap = cv.VideoCapture(rtsp_url)
    def read(self):
        return self.cap.read()
    def reconnect(self):
        self.cap.release()
        time.sleep(3)
        self.cap = cv.VideoCapture(self.rtsp_url)
    def is_opened(self):
        return self.cap.isOpened()
    def release(self):
        self.cap.release()
    def sett(self, exposure=-6, fps=30, width=1280, height=720):
        self.cap.set(cv.CAP_PROP_EXPOSURE, exposure)  # Lower is faster (e.g., -6, -7)
        self.cap.set(cv.CAP_PROP_FPS, fps)
        self.cap.set(cv.CAP_PROP_FRAME_WIDTH, width)
        self.cap.set(cv.CAP_PROP_FRAME_HEIGHT, height)
        
        
        
def detect_face_and_track_person():
    while not stop_event.is_set():
        try:
            frame = frame_queue.get(timeout=1) #it will wait upto 1 seconds
        except queue.Empty:
            continue
#         frame = frame_queue.get(timeout=1)
        results = ymodel.predict(source = frame, verbose = False)[0]
        detections = []
        for r in results.boxes.data.tolist():
            x1,y1,x2,y2,score,cls=r
            if int(cls)==0 and score>0.5:
                detections.append([x1,y1,x2,y2,score])
        tracked = []
        if detections:
            tracked = tracker.update(np.array(detections))
        current_ids = set()
        
        for d in tracked:
            x1,y1,x2,y2,track_id = [int(v) for v in d]
            current_ids.add(track_id)
            last_seen_frame[track_id] = frame_count
            person_crop = frame[y1:y2, x1:x2]
            if person_crop is None or person_crop.size == 0:
                continue
            rgb_crop = cv.cvtColor(person_crop, cv.COLOR_BGR2RGB)
            boxes,_ = detector.detect(rgb_crop)
            if len(boxes)>0:
                fx, fy, fw, fh = boxes[0]
                fx, fy = abs(fx), abs(fy)
                face_region = rgb_crop[fy:fy+fh, fx:fx+fw]
                face_region = cv.resize(face_region, (160, 160))
                if track_id not in already_recognized:
                    face_crop_queue.put((face_region,track_id))
            name = track_id_to_name.get(track_id, "Unknown")
            label = f"{name} | ID: {track_id}"
            cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv.putText(frame, label, (x1, y1 - 10),
            cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        for rid in list(already_recognized):
            if rid not in current_ids and frame_count - last_seen_frame.get(rid, 0) > inactive_threshold:
                already_recognized.remove(rid)
                track_id_to_name.pop(rid, None)
                logger.info(f"Removed ID {rid} due to inactivity.")
                
        cv.imshow('Face Recognition + Tracking', frame)
        if cv.waitKey(1) & 0xFF == ord('q'):
            stop_event.set()
            break


def face_recognition_and_logging():
    face_batch = []
    batch_id = []

    while not stop_event.is_set():
        try:
            face_region, track_id = face_crop_queue.get(timeout=1)
            face_batch.append(face_region)
            batch_id.append(track_id)

            if len(face_batch) >= BATCH_SIZE:
                embeddings = get_batch_embeddings(face_batch)

                for i, emb in enumerate(embeddings):
                    proba = model.predict_proba([emb])[0]
                    best_idx = np.argmax(proba)
                    confidence = proba[best_idx]

                    if confidence < 0.93:
                        predicted_name = "Unknown"
                    else:
                        predicted_name = encoder.inverse_transform([best_idx])[0]

                    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                    if predicted_name != "Unknown":
                        logger.info(f"{predicted_name} recognized with ID {batch_id[i]}")
                        newemployee.setdefault(predicted_name, []).append(now)
                        logger.info(f"logged for {predicted_name} at {now}")
                        track_id_to_name[batch_id[i]] = predicted_name
                        already_recognized.append(batch_id[i])

                        if batch_id[i] not in in_time_log:
                            in_time_log[batch_id[i]] = now
                        out_time_log[batch_id[i]] = now

                face_batch = []
                batch_id = []

        except queue.Empty:
            continue


def final_output():
    def calculate_total_duration(timestamps):
        total_seconds = 0
        for i in range(0, len(timestamps) - 1, 2):  # step by 2
            in_time = datetime.datetime.strptime(timestamps[i], "%Y-%m-%d %H:%M:%S")
            out_time = datetime.datetime.strptime(timestamps[i+1], "%Y-%m-%d %H:%M:%S")
            total_seconds += int((out_time - in_time).total_seconds())
        minutes = total_seconds // 60
        seconds = total_seconds % 60
        return minutes, seconds
    logger.info("Final attendance summary:")
    for employee, timestamps in newemployee.items():
        mins, secs = calculate_total_duration(timestamps)
        logger.info(f"{employee} stayed for {mins} min {secs} sec")

        

ext_cam = CameraStream(RTSP_URL)
detection_thread = Thread(target=detect_face_and_track_person)
recognition_thread = Thread(target=face_recognition_and_logging)

logger.info("Detection and recognition threads started.")
detection_thread.start()
recognition_thread.start()
try:
    while not stop_event.is_set():
        res, frame = ext_cam.read()
        if not res:
            logger.warning("Camera read failed, attempting to reconnect.")
            ext_cam.reconnect()
            continue

        # Resize and convert
        small_frame = cv.resize(frame, (320, 240))
        small_rgb = cv.cvtColor(small_frame, cv.COLOR_BGR2RGB)

        frame_queue.put(small_rgb)

        # Break on 'q'
        if cv.waitKey(1) & 0xFF == ord('q'):
            stop_event.set()
            break

except KeyboardInterrupt:
    logger.info("Interrupted by user")
    stop_event.set()


# Shutdown sequence
detection_thread.join()
recognition_thread.join()
ext_cam.release()
cv.destroyAllWindows()

# Final summary
print("i am hereeee")
final_output()


2025-05-02 23:32:06,851 | INFO | Detection and recognition threads started.


In [33]:
from facenet_pytorch import MTCNN
# Use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(keep_all=True, device=device)
img = cv.imread("image.jpg")

# Convert BGR (OpenCV default) to RGB (MTCNN expects RGB)
img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)

boxes,_ = detector.detect(img_rgb)
len(boxes)
a,b,c,d = boxes[0]
a

128.98880004882812

In [26]:
# testing
# img = cv.imread("image.jpg")
from mtcnn.mtcnn import MTCNN
detector = MTCNN()
img = cv.imread("image.jpg")

# Convert BGR (OpenCV default) to RGB (MTCNN expects RGB)
img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)

# Run face detection
faces = detector.detect_faces(img_rgb)
face = faces[1]
print(face['box'])
a,b,c,d = face['box']
a

[651, 131, 368, 490]


651

In [ ]:
array([[128.98880004882812, 37.547916412353516, 548.539794921875, 572.3052978515625],
        [647.2360229492188, 121.86314392089844, 1029.6724853515625, 605.4378051757812]], dtype=object),
 array([0.9923350214958191, 0.999972939491272], dtype=object)

In [47]:
from mtcnn.mtcnn import MTCNN
detector = MTCNN()
img = cv.imread("image.jpg")

# Convert BGR (OpenCV default) to RGB (MTCNN expects RGB)
img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
faces = detector.detect_faces(img_rgb)
results = ymodel.predict(source = img_rgb, verbose = False)
boxes = results.boxes.data.tolist()
for r in boxes:
    x1, y1, x2, y2, score, cls = box

AttributeError: 'list' object has no attribute 'boxes'

In [44]:

results.boxes

AttributeError: 'list' object has no attribute 'boxes'

In [60]:
import cv2 as cv
import numpy as np
from mtcnn.mtcnn import MTCNN
detector = MTCNN()
cap = cv.VideoCapture(r'video.mp4')
detections = []
while True:
    res,frame = cap.read()
    if not res:
        break
    results = ymodel.predict(source=frame, verbose=False)[0]
    detections = []
    for r in results.boxes.data.tolist():
        x1, y1, x2, y2, score, cls = r
        print(x1, y1, x2, y2, score, cls)
        if int(cls) == 0 and score > 0.5:  # cls 0 = person
            detections.append([x1, y1, x2, y2, score])
    cv.imshow("cap",frame)
    if cv.waitKey(1) & 0xFF ==ord('q'):
        break
cap.release()
cv.destroyAllWindows()

1460.0020751953125 469.442138671875 1543.877197265625 641.3814697265625 0.8196250796318054 0.0
1447.0364990234375 319.6604919433594 1516.2269287109375 480.1761474609375 0.7945894598960876 0.0
259.2726135253906 435.0619812011719 326.391845703125 609.52978515625 0.7787193655967712 0.0
664.6176147460938 649.043212890625 744.25 851.0556030273438 0.771995484828949 0.0
1141.4063720703125 950.4307250976562 1245.04443359375 1080.0 0.746316134929657 0.0
1793.9100341796875 144.2117156982422 1850.3973388671875 271.5205383300781 0.7450641393661499 0.0
750.1343994140625 450.0628356933594 822.37060546875 619.3534545898438 0.7428634166717529 0.0
1200.6650390625 126.65380096435547 1255.1890869140625 265.9782409667969 0.6848563551902771 0.0
198.33920288085938 44.102210998535156 266.801513671875 160.22470092773438 0.6116983294487 0.0
709.8422241210938 242.80357360839844 764.4305419921875 386.60247802734375 0.603485107421875 0.0
1138.5809326171875 504.7343444824219 1212.36181640625 654.0113525390625 0.59

1140.0545654296875 918.4625244140625 1242.1553955078125 1080.0 0.8554172515869141 0.0
1455.421630859375 328.18487548828125 1526.5760498046875 484.6392517089844 0.8410727381706238 0.0
673.163330078125 619.7933349609375 746.370361328125 798.041015625 0.792783260345459 0.0
755.2293701171875 430.3843688964844 822.0630493164062 585.6361694335938 0.7365195751190186 0.0
1396.9227294921875 703.2343139648438 1509.850830078125 886.6517944335938 0.7280511260032654 0.0
1202.628173828125 117.47731018066406 1254.7623291015625 249.958740234375 0.7248157858848572 0.0
40.25629806518555 803.1498413085938 143.54747009277344 990.9201049804688 0.7142650485038757 14.0
250.09561157226562 446.04473876953125 326.67108154296875 608.4327392578125 0.7117889523506165 0.0
1794.9027099609375 154.26736450195312 1855.5970458984375 286.3956298828125 0.6203590631484985 0.0
924.7913818359375 181.95526123046875 983.1873779296875 319.0350646972656 0.5963059663772583 0.0
799.6478271484375 71.7330322265625 841.7950439453125 

1139.9647216796875 476.3963928222656 1214.013427734375 612.9448852539062 0.8701664209365845 14.0
674.8305053710938 604.1713256835938 747.7985229492188 794.992919921875 0.8081998229026794 0.0
1461.57421875 339.8868713378906 1532.208984375 502.8776550292969 0.7594785690307617 0.0
246.5888671875 462.6602783203125 327.0555725097656 637.5997924804688 0.7457079887390137 0.0
1850.4371337890625 649.5465087890625 1919.5338134765625 842.161376953125 0.7058569192886353 0.0
1137.6611328125 894.89208984375 1238.9501953125 1080.0 0.685798168182373 0.0
1394.4332275390625 708.0684814453125 1498.6278076171875 889.147705078125 0.6433421969413757 0.0
757.1746215820312 421.0660705566406 814.2669067382812 576.8116455078125 0.6298922896385193 0.0
1793.2364501953125 156.13677978515625 1862.4803466796875 286.78045654296875 0.6131987571716309 0.0
1621.365966796875 17.212806701660156 1661.073486328125 127.53861236572266 0.5994924306869507 0.0
1473.5428466796875 492.4550476074219 1553.615234375 659.4050903320312

675.1375122070312 595.4009399414062 750.954345703125 793.44091796875 0.8359924554824829 0.0
1134.5413818359375 880.8633422851562 1231.74462890625 1073.0836181640625 0.8132540583610535 0.0
1394.9794921875 709.7842407226562 1486.7193603515625 887.2734985351562 0.753127932548523 0.0
1848.8836669921875 645.0992431640625 1919.4730224609375 837.97119140625 0.75239098072052 0.0
245.64450073242188 470.73321533203125 326.4185485839844 648.5960083007812 0.7464244961738586 0.0
1462.40380859375 351.0127258300781 1535.465087890625 517.8353271484375 0.7405228018760681 0.0
1796.6337890625 157.2360076904297 1862.136474609375 287.8872375488281 0.7041687965393066 0.0
925.9588623046875 191.3385772705078 980.4364013671875 323.1672668457031 0.6849389672279358 0.0
373.1336669921875 4.578758239746094 425.3722229003906 123.49980926513672 0.605695366859436 0.0
1620.4405517578125 17.18807601928711 1662.3519287109375 125.11222076416016 0.6048572063446045 0.0
754.8507080078125 414.41143798828125 812.9749755859375

1470.352294921875 359.033203125 1543.67138671875 518.7073974609375 0.8505598902702332 0.0
1128.1417236328125 862.8579711914062 1221.7646484375 1071.945556640625 0.8088200092315674 0.0
674.6810913085938 577.24951171875 746.9711303710938 772.656005859375 0.7687186598777771 0.0
1483.47216796875 514.3894653320312 1563.9222412109375 692.9332275390625 0.7681750655174255 0.0
1837.7120361328125 633.4031982421875 1919.45068359375 833.14453125 0.7415053248405457 0.0
1383.119384765625 711.17041015625 1467.845947265625 883.6052856445312 0.7281190752983093 0.0
242.22720336914062 480.5743103027344 323.2213439941406 655.2767944335938 0.7242394685745239 0.0
52.59100341796875 817.5256958007812 176.52499389648438 986.333740234375 0.7123304009437561 14.0
754.22265625 401.95111083984375 812.0105590820312 565.3005981445312 0.7106810212135315 0.0
1801.6871337890625 165.4136505126953 1862.6495361328125 298.0065002441406 0.7105175256729126 0.0
925.3441772460938 195.74354553222656 978.1920776367188 334.6501770

...

107.71666717529297 850.6138916015625 200.56788635253906 1060.35205078125 0.8345445394515991 0.0
1034.4632568359375 886.21875 1117.9205322265625 1078.5142822265625 0.775808572769165 0.0
673.2564697265625 878.7689208984375 762.7075805664062 1080.0 0.7735786437988281 0.0
1638.5482177734375 354.2200012207031 1707.2347412109375 519.3939208984375 0.7677335143089294 0.0
1186.464599609375 809.2694091796875 1278.651611328125 1026.46728515625 0.7663270235061646 0.0
910.76513671875 368.400146484375 981.9840087890625 523.080322265625 0.7590066194534302 0.0
1827.70166015625 360.9998474121094 1908.598388671875 519.4671630859375 0.7360414862632751 0.0
1053.3292236328125 492.5583801269531 1130.370849609375 655.234375 0.7074856758117676 0.0
551.9805908203125 481.8525085449219 637.6312866210938 636.0999755859375 0.6864094138145447 0.0
189.680908203125 49.2948112487793 244.75868225097656 152.3103485107422 0.6796672344207764 0.0
1007.6621704101562 706.6463623046875 1063.1202392578125 883.4945678710938 0.6

103.72478485107422 867.6802368164062 201.67283630371094 1080.0 0.8352712392807007 0.0
1027.6900634765625 896.7725830078125 1108.5164794921875 1080.0 0.8285302519798279 0.0
673.0223388671875 863.4116821289062 764.1694946289062 1069.8883056640625 0.7988124489784241 0.0
1187.7723388671875 789.2843627929688 1278.9779052734375 991.7382202148438 0.7957948446273804 0.0
910.2064819335938 372.5083923339844 980.1533203125 526.3942260742188 0.77461838722229 0.0
23.973094940185547 576.263671875 135.7723846435547 750.9508666992188 0.7575944066047668 0.0
1671.94482421875 935.2527465820312 1782.7357177734375 1080.0 0.7275338172912598 0.0
1667.502685546875 762.1409912109375 1775.942138671875 951.1571044921875 0.7269700765609741 0.0
1763.8001708984375 263.76812744140625 1841.75732421875 417.353515625 0.7244656682014465 0.0
1630.4981689453125 347.7295227050781 1702.5721435546875 504.61944580078125 0.7125164270401001 0.0
552.8095703125 486.2846374511719 641.4168701171875 651.0489501953125 0.6771017313003

550.6331176757812 498.06842041015625 641.9422607421875 658.5098876953125 0.7986811399459839 0.0
1617.0428466796875 336.5247802734375 1697.8970947265625 501.06298828125 0.7915013432502747 0.0
675.77392578125 840.4813842773438 764.8382568359375 1050.6243896484375 0.7897233963012695 0.0
1017.4347534179688 913.6948852539062 1095.5902099609375 1080.0 0.7766069769859314 0.0
1051.714599609375 470.1927795410156 1129.9610595703125 644.6689453125 0.7727660536766052 0.0
909.8800048828125 385.8237609863281 977.9058837890625 548.0679931640625 0.7687846422195435 0.0
34.83566665649414 587.3051147460938 133.27684020996094 775.691162109375 0.7687670588493347 0.0
95.60533905029297 893.311767578125 200.7164764404297 1080.0 0.7650319933891296 0.0
1762.434814453125 272.5782775878906 1841.45654296875 436.26031494140625 0.7555760741233826 0.0
1190.5169677734375 766.6257934570312 1279.6358642578125 950.3549194335938 0.7363874912261963 0.0
1003.2317504882812 679.9951171875 1062.434814453125 852.8106079101562 0

83.63733673095703 908.1699829101562 195.46275329589844 1080.0 0.8071979880332947 0.0
1001.6045532226562 930.5275268554688 1082.8238525390625 1077.8687744140625 0.7922354936599731 0.0
677.125 825.4521484375 768.0532836914062 1051.243896484375 0.77669757604599 0.0
1616.7711181640625 327.26641845703125 1692.028564453125 498.2621154785156 0.7737030982971191 0.0
1191.9127197265625 749.08837890625 1279.955322265625 949.966796875 0.7423520088195801 0.0
908.0985717773438 395.90350341796875 976.5078735351562 557.6350708007812 0.7373271584510803 0.0
43.96350860595703 589.5646362304688 136.7105712890625 768.3746337890625 0.7250638008117676 0.0
1056.1103515625 459.43707275390625 1132.3675537109375 638.7627563476562 0.7246792912483215 0.0
1761.072509765625 279.44439697265625 1842.402099609375 443.4661560058594 0.7246496677398682 0.0
1112.218994140625 208.52581787109375 1162.607421875 334.81201171875 0.6900942921638489 0.0
998.70361328125 664.71484375 1066.2747802734375 845.4296875 0.678185701370239

990.9684448242188 935.6687622070312 1075.5037841796875 1080.0 0.8347032070159912 0.0
1612.4970703125 322.71588134765625 1683.54638671875 486.87286376953125 0.801731288433075 0.0
1198.07421875 734.4210815429688 1285.2822265625 938.1868896484375 0.7991015315055847 0.0
77.08493041992188 920.7777099609375 185.74440002441406 1080.0 0.763554573059082 0.0
1761.45263671875 285.59912109375 1843.864013671875 442.1452331542969 0.7529333233833313 0.0
548.3121337890625 513.3773803710938 627.2803955078125 664.849609375 0.7517120838165283 0.0
49.311378479003906 595.5994873046875 138.99827575683594 774.33984375 0.7345395684242249 0.0
683.4863891601562 810.6897583007812 771.1705322265625 1046.68701171875 0.733860433101654 0.0
906.5575561523438 400.1018371582031 973.6221313476562 557.7119750976562 0.7193244695663452 0.0
1059.23974609375 450.1285400390625 1131.9619140625 619.0625610351562 0.6728256940841675 0.0
1171.6142578125 231.14044189453125 1224.3782958984375 357.95452880859375 0.6520167589187622 0.

In [6]:
import cv2 as cv
import numpy as np
from mtcnn.mtcnn import MTCNN
detector = MTCNN()
cap = cv.VideoCapture(r'stairs.mp4')
detections = []
while True:
    res,frame = cap.read()
    if not res:
        break
    results = ymodel.predict(source=frame, verbose=False)[0]
    detections = []
    for r in results.boxes.data.tolist():
        x1, y1, x2, y2, score, cls = r
        print(x1, y1, x2, y2, score, cls)
        if int(cls) == 0 and score > 0.5:  # cls 0 = person
            detections.append([x1, y1, x2, y2, score])
    cv.imshow("cap",frame)
    if cv.waitKey(1) & 0xFF ==ord('q'):
        break
cap.release()
cv.destroyAllWindows()
# https://chatgpt.com/share/6814ba31-6dcc-800d-b32a-761b53225311

78.91317749023438 167.45343017578125 360.522216796875 715.9005126953125 0.8964690566062927 0.0
0.06850433349609375 486.791259765625 79.9459457397461 716.0201416015625 0.34894606471061707 0.0
79.83587646484375 169.9384765625 361.327880859375 715.7720947265625 0.8979068994522095 0.0
0.077911376953125 485.3138427734375 81.4714584350586 716.6929931640625 0.4061254858970642 0.0
77.9339599609375 170.1190185546875 360.7747802734375 716.15380859375 0.9052211046218872 0.0
0.07625579833984375 485.970703125 81.9613265991211 716.791015625 0.41393840312957764 0.0
79.54888916015625 171.9957275390625 361.78173828125 717.269775390625 0.9119510054588318 0.0
0.074951171875 486.0570983886719 83.09852600097656 717.063232421875 0.4296925663948059 0.0
156.78115844726562 378.9820556640625 333.533203125 504.0877685546875 0.27899742126464844 63.0
79.81460571289062 172.859619140625 363.8875732421875 716.3958129882812 0.9104012250900269 0.0
0.07721710205078125 486.35443115234375 82.89302825927734 717.06231689453

38.25616455078125 194.05633544921875 363.234619140625 716.7113647460938 0.8934701681137085 0.0
0.081298828125 486.5280456542969 96.36893463134766 717.83544921875 0.41870352625846863 0.0
115.78671264648438 416.455810546875 280.5382385253906 541.496337890625 0.35612753033638 63.0
311.34783935546875 155.13304138183594 344.52081298828125 211.40599060058594 0.25637462735176086 0.0
37.2900390625 195.3717041015625 367.3885498046875 717.3055419921875 0.8963175415992737 0.0
0.07707977294921875 486.6971435546875 95.69205474853516 717.868408203125 0.41436973214149475 0.0
311.3309020996094 155.0916748046875 344.5327453613281 211.3345947265625 0.26151397824287415 0.0
37.088897705078125 196.6165771484375 363.9429626464844 717.8101806640625 0.8849579691886902 0.0
0.05657196044921875 481.218017578125 91.63320922851562 715.0523681640625 0.3971013128757477 0.0
311.28009033203125 155.18560791015625 344.56365966796875 211.3031005859375 0.26422426104545593 0.0
34.8154296875 197.796630859375 364.81689453125

30.250030517578125 197.85577392578125 369.9757385253906 717.2845458984375 0.8754170536994934 0.0
0.0 480.30328369140625 93.57931518554688 715.5908813476562 0.31949180364608765 0.0
30.0626220703125 197.8607177734375 370.31170654296875 717.288330078125 0.8733603954315186 0.0
0.0 480.14794921875 94.2711181640625 715.4293212890625 0.3161619007587433 0.0
30.05145263671875 197.72186279296875 370.219482421875 717.4111328125 0.8709582090377808 0.0
0.0 479.9478759765625 94.48756408691406 715.457763671875 0.31796541810035706 0.0
30.014862060546875 197.6839599609375 370.2422180175781 717.6143798828125 0.8628681302070618 0.0
0.0 480.0999755859375 94.94320678710938 715.46533203125 0.31891968846321106 0.0
30.0284423828125 197.77783203125 370.2242431640625 717.576171875 0.8644500970840454 0.0
0.0 479.94708251953125 95.68766021728516 715.5112915039062 0.31657710671424866 0.0
30.07769775390625 197.7078857421875 370.2353515625 717.518798828125 0.8681054711341858 0.0
0.0 479.4270324707031 95.577987670898

80.50091552734375 172.99627685546875 354.65802001953125 717.4722290039062 0.9266835451126099 0.0
218.4293212890625 334.0465087890625 313.84564208984375 416.3673095703125 0.26216432452201843 63.0
85.55633544921875 169.60162353515625 354.94696044921875 718.0087280273438 0.9284577369689941 0.0
209.52113342285156 326.6217041015625 319.297119140625 409.3623046875 0.4670374393463135 63.0
91.68753051757812 166.18206787109375 355.2347106933594 717.195068359375 0.9100143909454346 0.0
95.62408447265625 164.4290771484375 354.8426513671875 715.9482421875 0.8969582319259644 0.0
99.8497314453125 164.25006103515625 354.7935485839844 717.5253295898438 0.8856369853019714 0.0
104.49560546875 163.0526123046875 355.1544494628906 716.5826416015625 0.8899336457252502 0.0
103.94933319091797 160.8931884765625 355.2200927734375 717.806640625 0.8537497520446777 0.0
103.86810302734375 157.0384521484375 353.24468994140625 716.5008544921875 0.8501445055007935 0.0
107.76409912109375 158.14697265625 354.971069335937

218.99435424804688 193.91871643066406 464.1141662597656 628.0712890625 0.9146770238876343 0.0
345.0489501953125 270.46685791015625 442.3668212890625 363.58038330078125 0.48180994391441345 63.0
224.7937774658203 197.9821319580078 462.04571533203125 629.6688842773438 0.899397611618042 0.0
347.96856689453125 272.72674560546875 445.43865966796875 362.9444580078125 0.500957727432251 63.0
350.599365234375 273.0679931640625 444.639404296875 358.322509765625 0.31164488196372986 62.0
227.13497924804688 201.55535888671875 465.1198425292969 629.5845336914062 0.8995388150215149 0.0
351.77984619140625 277.0231018066406 448.4725341796875 367.3749084472656 0.5457925796508789 63.0
352.64239501953125 275.6082763671875 448.5732421875 364.6099853515625 0.4619041085243225 62.0
232.59115600585938 207.430908203125 462.7123107910156 627.86181640625 0.8886473774909973 0.0
358.31146240234375 279.48077392578125 452.08819580078125 368.34393310546875 0.5964475870132446 63.0
358.384033203125 278.9898681640625 451.

316.360107421875 253.89715576171875 497.283203125 612.2807006835938 0.8051218390464783 0.0
404.98956298828125 297.9696044921875 521.2079467773438 399.88177490234375 0.3965897560119629 63.0
320.3572998046875 257.8798828125 506.89642333984375 610.0914916992188 0.7947589755058289 0.0
412.58465576171875 299.05047607421875 527.1072998046875 400.7716064453125 0.3810422718524933 63.0
331.45654296875 262.306884765625 515.7032470703125 612.75439453125 0.7767438292503357 0.0
420.2869873046875 299.0335693359375 530.7237548828125 400.86767578125 0.4074018597602844 63.0
333.78350830078125 263.36083984375 523.075927734375 608.91552734375 0.7849718332290649 0.0
424.1944580078125 302.2259521484375 536.4259643554688 401.635986328125 0.4325244724750519 63.0
336.1846923828125 265.248779296875 516.9051513671875 606.6445922851562 0.7775167226791382 0.0
427.43121337890625 302.96649169921875 539.5444946289062 401.48004150390625 0.462821900844574 63.0
335.67608642578125 265.459228515625 511.63104248046875 603

509.19573974609375 301.01544189453125 619.1566772460938 480.89373779296875 0.2944415211677551 0.0
506.952880859375 289.8804931640625 755.467041015625 632.2872314453125 0.2696056663990021 0.0
512.7227783203125 303.9620056152344 624.2220458984375 484.6871032714844 0.2594381272792816 0.0
392.30377197265625 298.29119873046875 753.3899536132812 635.5982055664062 0.29092779755592346 0.0
393.56365966796875 305.47125244140625 757.1265258789062 637.5137329101562 0.3514711260795593 0.0
399.1619567871094 309.4117431640625 758.2769775390625 635.1732177734375 0.5366013646125793 0.0
533.5133666992188 313.7592468261719 599.3689575195312 394.6143493652344 0.25703299045562744 0.0
393.22174072265625 310.08599853515625 757.2904663085938 635.044921875 0.4301527440547943 0.0
538.3079833984375 316.75506591796875 603.8599853515625 396.38580322265625 0.33723872900009155 0.0
396.2156982421875 312.20245361328125 756.5469970703125 635.4849853515625 0.34895023703575134 0.0
545.595458984375 318.00750732421875 606.

395.02630615234375 263.12298583984375 587.2981567382812 536.7216796875 0.35435035824775696 0.0
472.38189697265625 263.22113037109375 571.5623168945312 403.49114990234375 0.3424609899520874 0.0
392.5712890625 263.91766357421875 583.1549072265625 534.2739868164062 0.3430885970592499 0.0
388.398193359375 265.0372314453125 576.4766845703125 535.79248046875 0.47914785146713257 0.0
392.173095703125 267.2733154296875 575.9613647460938 536.5531005859375 0.5272594094276428 0.0
383.816162109375 266.29833984375 563.1847534179688 581.1527709960938 0.530727207660675 0.0
383.04205322265625 266.8358154296875 562.4305419921875 589.8477783203125 0.5463081002235413 0.0
382.76995849609375 265.281982421875 561.9346313476562 586.6604614257812 0.5103517770767212 0.0
389.4439697265625 265.2601318359375 567.786376953125 599.7484130859375 0.5339346528053284 0.0
392.44573974609375 263.34844970703125 576.8340454101562 601.8740234375 0.43838435411453247 0.0
396.12060546875 261.96331787109375 561.4247436523438 571

203.76068115234375 179.23060607910156 405.66156005859375 639.3582763671875 0.896040141582489 0.0
201.97640991210938 181.468017578125 403.2229309082031 655.0852661132812 0.8918692469596863 0.0
201.0931396484375 180.93699645996094 405.93072509765625 672.081298828125 0.8953062891960144 0.0
200.28578186035156 181.5406494140625 405.0411376953125 673.103271484375 0.9023187160491943 0.0
199.33859252929688 181.7945098876953 404.2478332519531 673.0932006835938 0.8984516859054565 0.0
196.66038513183594 180.97296142578125 403.5177001953125 672.569091796875 0.89601069688797 0.0
197.05020141601562 175.0575714111328 403.8031921386719 675.1400756835938 0.8931102752685547 0.0
194.58316040039062 166.3605194091797 403.5866394042969 674.3463745117188 0.8920374512672424 0.0
193.59933471679688 157.561767578125 403.1665954589844 674.52587890625 0.8858082890510559 0.0
190.77566528320312 149.8089599609375 403.1446228027344 672.0564575195312 0.8934047818183899 0.0
189.60003662109375 144.227294921875 402.478942